In [ ]:
#connect to Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
#loading demographics dataset
pd.read_csv('/content/gdrive/My Drive/training_data/user_demographics_v1.csv')

,uid,reg_date,device,gender,country,age
0,54030035.0,2017-06-29T00:00:00Z,and,M,USA,19
1,72574201.0,2018-03-05T00:00:00Z,iOS,F,TUR,22
2,64187558.0,2016-02-07T00:00:00Z,iOS,M,USA,16
3,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41
4,99231338.0,2017-03-26T00:00:00Z,iOS,M,FRA,59
...,...,...,...,...,...,...
9995,43138741.0,2016-11-23T00:00:00Z,and,M,BRA,16
9996,30326568.0,2016-08-21T00:00:00Z,and,M,BRA,15
9997,92384369.0,2015-08-20T00:00:00Z,and,F,DEU,33
9998,98809962.0,2017-04-08T00:00:00Z,iOS,F,FRA,32


In [ ]:
#loading customer subscription dataset
pd.read_csv('/content/gdrive/My Drive/training_data/purchase_data_v1.csv')

,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599
...,...,...,...,...
9001,2017-02-06,16377492,sku_three_499,499
9002,2017-03-01,16377492,sku_seven_1499,499
9003,2017-05-29,92513925,sku_two_299,299
9004,2017-08-23,92513925,sku_four_599,599


In [ ]:
customer_demographics = pd.read_csv('/content/gdrive/My Drive/training_data/user_demographics_v1.csv')
customer_subscrptions = pd.read_csv('/content/gdrive/My Drive/training_data/purchase_data_v1.csv')

In [ ]:
#print columns of dataset
print(customer_demographics.columns)

Index(['uid', 'reg_date', 'device', 'gender', 'country', 'age'], dtype='object')


In [ ]:
print(customer_subscrptions.columns)

Index(['date', 'uid', 'sku', 'price'], dtype='object')


In [ ]:
#merge customer_demographics (Left) and customer_subscriptions (right)
sub_data_demo = customer_demographics.merge(
    #merger right dataframe
    customer_subscrptions,
    #join type (inner,outer,left,right)
    how='inner',
    #match the column
    on=['uid']
)
sub_data_demo.head()

,uid,reg_date,device,gender,country,age,date,sku,price
0,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2017-10-20,sku_three_499,499
1,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2017-05-29,sku_two_299,299
2,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2017-08-23,sku_four_599,599
3,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41,2018-03-26,sku_six_1299,299
4,16377492.0,2016-10-16T00:00:00Z,and,M,BRA,20,2018-03-17,sku_one_199,199


In [ ]:
#combine demographics and purchase data
sub_data_grp = sub_data_demo.groupby(by=['country','device'],
                                     axis=0,
                                     as_index=False
                                     )
sub_data_grp

In [ ]:
#mean price paid for each country/device
sub_data_grp.price.mean()

,country,device,price
0,BRA,and,412.985594
1,BRA,iOS,404.739300
2,CAN,and,406.826087
3,CAN,iOS,386.573964
4,DEU,and,402.474903
5,DEU,iOS,417.639798
6,FRA,and,418.377163
7,FRA,iOS,382.921569
8,TUR,and,433.913793
9,TUR,iOS,390.176471


In [ ]:
# mean & median price paid for each country/device
sub_data_grp.price.agg(['mean','median'])

mean  median
country device                    
BRA     and     412.985594   499.0
        iOS     404.739300   299.0
CAN     and     406.826087   499.0
        iOS     386.573964   299.0
DEU     and     402.474903   299.0
        iOS     417.639798   499.0
FRA     and     418.377163   499.0
        iOS     382.921569   299.0
TUR     and     433.913793   499.0
        iOS     390.176471   299.0
USA     and     399.000000   299.0
        iOS     410.684783   499.0

In [ ]:
#calculate multiple metrics across different groups
sub_data_grp.agg({'price':['mean','min','max'],
                  'age':['mean','min','max']})

country device       price                 age        
                         mean min  max       mean min max
0      BRA    and  412.985594  99  899  23.913565  15  62
1      BRA    iOS  404.739300  99  899  23.975681  15  66
2      CAN    and  406.826087  99  899  23.282609  15  49
3      CAN    iOS  386.573964  99  899  25.023669  15  57
4      DEU    and  402.474903  99  899  23.814672  15  66
5      DEU    iOS  417.639798  99  899  20.773300  15  42
6      FRA    and  418.377163  99  899  23.200692  15  55
7      FRA    iOS  382.921569  99  899  24.407843  15  59
8      TUR    and  433.913793  99  899  24.004310  15  56
9      TUR    iOS  390.176471  99  899  22.591176  15  51
10     USA    and  399.000000  99  899  24.564738  15  65
11     USA    iOS  410.684783  99  899  24.144022  15  75

In [ ]:
#Conversion rate: maximum lapse date
from datetime import datetime,timedelta

current_date =pd.to_datetime('2018-03-17')

#Lapse date = the date trial ends for a given user
#lapse_date = pd.to_datetime(sub_data_demo.reg_date) + timedelta(days=7) 
print(sub_data_demo.reg_date.max())

2018-03-17T00:00:00Z


In [ ]:
#set a week before today
max_lapse_date = current_date -timedelta(days=7)
#restrict users lapsed before max_lapse_date
conv_sub_data = sub_data_demo[(pd.to_datetime(sub_data_demo.reg_date).dt.date < max_lapse_date.date())]
#count the users remaining in our data
total_user_count = conv_sub_data.price.count()
print(total_user_count)

8988


In [ ]:
#latest subscription date: within 7 days of lapsing
max_sub_date = pd.to_datetime(conv_sub_data.date) + timedelta(days=7)

#filter the users with non-zero subscription price who subscribe before max_sub_date

total_subs = conv_sub_data[
                           (conv_sub_data.price > 0) & (conv_sub_data.date <= max_sub_date)
]
#count the users remaining in our data
total_subs_count = total_subs.price.count()
print(total_subs_count)

8988


In [ ]:
#calculate the conversion rate
conversion_rate = total_subs_count / total_user_count
print(conversion_rate)

1.0
